In [4]:
import numpy as np
import cv2

In [5]:
def estimate_optical_flow(image1, image2):
    # Convert images to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Parameters for the Lucas-Kanade optical flow method
    lk_params = dict(winSize=(15, 15),
                     maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Calculate optical flow using Lucas-Kanade method
    optical_flow = cv2.calcOpticalFlowPyrLK(gray1, gray2, None, None, **lk_params)

    # Select good points
    good_points = optical_flow[0] is not None

    # Filter out the good points
    good_old = optical_flow[0][good_points]
    good_new = optical_flow[1][good_points]

    # Calculate the motion vectors
    motion_vectors = good_new - good_old

    return good_points, good_old, motion_vectors

In [6]:
def visualize_optical_flow(image1, good_old, motion_vectors):
    # Create an output image as a copy of the original
    output_image = image1.copy()

    # Scale the motion vectors for better visualization
    scale_factor = 5
    scaled_motion_vectors = motion_vectors * scale_factor

    # Draw motion vectors on the output image
    for point, vector in zip(good_old, scaled_motion_vectors):
        point1 = tuple(np.round(point).astype(int))
        point2 = tuple(np.round(point + vector).astype(int))
        cv2.arrowedLine(output_image, point1, point2, (0, 255, 0), 2)

    # Display the result
    cv2.imshow('Optical Flow Visualization', output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [7]:
frame1 = cv2.imread("/home/logan_20/kitti_slam/dataset/sequences/00/image_0/000000.png")
frame2 = cv2.imread("/home/logan_20/kitti_slam/dataset/sequences/00/image_0/000001.png")

good_points, good_old, motion_vectors = estimate_optical_flow(frame1, frame2)

# Visualize the motion vectors on the first frame
if good_points.any():
    visualize_optical_flow(frame1, good_old, motion_vectors)
else:
    print("No good points found for optical flow.")

error: OpenCV(4.7.0) /io/opencv/modules/video/src/lkpyramid.cpp:1260: error: (-215:Assertion failed) (npoints = prevPtsMat.checkVector(2, CV_32F, true)) >= 0 in function 'calc'
